In [2]:
from lib import run_definition
from lib import run_result_location
from lib import constants
from lib import run
from os import path
import json
import glob
import numpy as np
import pandas as pd
from jsonpath_ng import jsonpath, parse
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


%matplotlib inline
#global defaults of graphics size
plt.rcParams["figure.figsize"] = [50, 50]

#Define run_id
#run_id = "2020-10-23-15-59-41"
run_id = "2020-10-23-16-10-36"

#point to test result folder containing runs
#location = perfmodel.RunResultLocation("../testresults/azure.1-auto-standalone")
location = run_result_location.RunResultLocation("../testresults/aws.1-auto-standalone")

runDefinition = run_definition.RunDefinition(location)
runDefinition.process_run_samples()
run = runDefinition.find_run(run_id)
meta = run.run_meta
df_latencies = pd.DataFrame(data={'run1': run.export_latency_node_distinct_latencies()})
df_broker_latencies = pd.DataFrame(data={'run1': run.export_broker_node_distinct_latencies()})
df_ping = pd.DataFrame(run.export_ping_series(constants.c_ping_all_metrics))

ImportError: cannot import name 'Run' from partially initialized module 'lib.run' (most likely due to a circular import) (/Users/jt/myproject/solace/azure-performance/solace-azure-performance/github/az-use-case-perf-tests/analytics/notebooks/lib/run.py)

## Single Detailed Run Analytics

CONFIDENTIAL | NOT FOR DISTRIBUTION | DATA NOT VALIDATED

Cloudprovider: {{meta.cloud_provider}}

Infrastructure: {{meta.infrastructure}}

Run name: {{meta.run_name}}

Run Id: {{meta.run_id}}

Started: {{meta.ts_run_start}}

Ended: {{meta.ts_run_end}}

Duration: {{meta.run_duration_sec()}}


# Key Metrics

### Latency Node Basic Stats

In [ ]:
run1_series = df_latencies['run1']
run1_series.describe()

### Broker Node Basic Stats

In [ ]:
broker_run1_series = df_broker_latencies['run1']
broker_run1_series.describe()

### Latency Node Quantiles

In [ ]:
q1 = run1_series.quantile(q=[0.05,0.1,0.25,0.5,0.75,0.9,0.99,0.995,0.999,0.9999,0.99995,0.99999])
q1

### Broker Node Quantiles

In [ ]:
broker_q1 = broker_run1_series.quantile(q=[0.05,0.1,0.25,0.5,0.75,0.9,0.99,0.995,0.999,0.9999,0.99995,0.99999])
broker_q1






In [ ]:
plt.figure(figsize=(25,15))
sns.displot(data=df_latencies)

In [ ]:
plt.figure(figsize=(25,15))
sns.displot(data=df_broker_latencies)


## DELTA Series

In [ ]:
delta_index_latencies = run.export_latency_node_delta_index_latencies(lambda latency: latency >= q1.get(0.99))
df_delta_index_latencies = pd.DataFrame(data={"delta":delta_index_latencies})
df_delta_index_latencies['delta'].describe()

In [ ]:
plt.figure(figsize=(25,15))
sns.displot(data=df_delta_index_latencies)

In [ ]:
lat_series_length = run.export_latency_node_series_length_latencies(lambda latency: latency >= q1.get(0.99))
s = lat_series_length[constants.k_latency_series_length]
g = lat_series_length[constants.k_latency_gap_length]
df_series_length = pd.DataFrame(data={"series_length": s})
df_series_length = pd.DataFrame(data={"gap_length": g})
df_series_length